# Environment Setup

In [4]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer,*-*"

Installed Packages Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.23258.3

In [23]:
#!connect mssql --kernel-name AGDemo-1a "Persist Security Info=False; TrustServerCertificate=True; Integrated Security=true; Initial Catalog=master; Server=rp-sql19ags-1a.perf.rubrik.com;"
#!connect mssql --kernel-name AGDemo-1b "Persist Security Info=False; TrustServerCertificate=True; Integrated Security=true; Initial Catalog=master; Server=rp-sql19ags-1b.perf.rubrik.com;"
#!connect mssql --kernel-name AGDemo-1c "Persist Security Info=False; TrustServerCertificate=True; Integrated Security=true; Initial Catalog=master; Server=rp-sql19ags-1c.perf.rubrik.com;"

Error: System.Exception: Login failed. The login is from an untrusted domain and cannot be used with Integrated authentication.
   at Microsoft.DotNet.Interactive.SqlServer.MsSqlKernel.ConnectAsync()
   at Microsoft.DotNet.Interactive.SqlServer.MsSqlKernelConnector.CreateKernelAsync(String kernelName)
   at Microsoft.DotNet.Interactive.SqlServer.ConnectMsSqlCommand.ConnectKernelsAsync(KernelInvocationContext context, InvocationContext commandLineContext)
   at Microsoft.DotNet.Interactive.CompositeKernel.<>c__DisplayClass21_0.<<AddKernelConnector>b__0>d.MoveNext() in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\CompositeKernel.cs:line 291
--- End of stack trace from previous location ---
   at System.CommandLine.NamingConventionBinder.CommandHandler.GetExitCodeAsync(Object returnValue, InvocationContext context)
   at System.CommandLine.NamingConventionBinder.ModelBindingCommandHandler.InvokeAsync(InvocationContext context)
   at System.CommandLine.Invocation.InvocationPipeline.<>c_

In [14]:
# Rubrik Connection Information
$ServiceAccountPath = '/Users/Steven.Tong/OneDrive - Rubrik Inc/git/rsc-service-account-rr.json'
$ServiceAccountContent = Get-Content -Path "$serviceAccountPath" -ErrorAction Stop | ConvertFrom-Json

$Server.access_token_uri
$ServiceAccountID = $ServiceAccountContent.client_id
$Secret = $ServiceAccountContent.client_secret

# Primary SQL Server
$PrimarySQLServerInstance = 'rp-sql19ags-1c.perf.rubrik.com'

# Secondary SQL Servers
$SecondarySQLServerInstance = 'rp-sql19ags-1a.perf.rubrik.com', 'rp-sql19ags-1b.perf.rubrik.com'

# Availability Group Name
$AvailabilityGroupName = 'rp-sql19ags-g1'
# Source Database Name
$DatabaseName = 'ProductionDatabase'
# Target Database Name
$TargetDatabaseName = 'TongExport'

# Append the following to the existing filenames to avoid conflicts
$FilenameAppend = '_tongexport'

# Connect to Rubrik

In [15]:
Connect-RubrikSecurityCloud -ServiceAccountPath $ServiceAccountPath

Connected!


# Refresh the Hosts of the SQL Servers in RSC

In [16]:
$QueryParms = @{
    "hostname" = "$($PrimarySQLServerInstance)"
    "instance" = "MSSQLSERVER"
}
$RSCMSQLInstance = Invoke-RubrikQuery -Path ./GetMSSQLInstance.gql -QueryParams $QueryParms


$QueryParms = @{
    "id" = "$($RSCMSQLInstance.id)"
}
 Invoke-RubrikQuery -Path ./PhysicalHostRefreshMutation.gql -QueryParams $QueryParms

foreach ($SQLInstance in $SecondarySQLServerInstance){
    $QueryParms = @{
        "hostname" = "$($SQLInstance)"
        "instance" = "MSSQLSERVER"
    }
    $RSCMSQLInstance = Invoke-RubrikQuery -Path ./GetMSSQLInstance.gql -QueryParams $QueryParms

    $QueryParms = @{
        "id" = "$($RSCMSQLInstance.id)"
    }
    Invoke-RubrikQuery -Path ./PhysicalHostRefreshMutation.gql -QueryParams $QueryParms
}
Start-Sleep -Seconds 45

# Get the ID of the Availability Group

In [11]:
Write-Host "Getting ID of AG: $AvailabilityGroupName"

$QueryParms = @{
    "AvailabilityGroupName" = $($AvailabilityGroupName)
}
$RSCMSSQLAvailabilityGroup = Invoke-RubrikQuery -Path ./GetMSSQLAvailabilityGroup.gql -QueryParams $QueryParms

if ($RSCMSSQLAvailabilityGroup.id -eq $null) {
    Write-Error "No AG found with name: $AvailabilityGroupName"
}

if ($RSCMSSQLAvailabilityGroup.count -gt 1) {
    Write-Error "Multiple AG found with name: $AvailabilityGroupName"
}
$RSCMSSQLAvailabilityGroup


id                                   name           cluster
--                                   ----           -------
37d606ef-0e98-5e15-bb93-b50497484694 rp-sql19ags-g1 @{id=39b92c18-d897-4b55-a7f9-17ff178616d0; nam…



# Get the ID of the Database

In [17]:
$QueryParms = @{
    "fid" = "$($RSCMSSQLAvailabilityGroup.id)"
    "DatabaseName" = "$($DatabaseName)"
}
$RSCMSSQLDatabase = Invoke-RubrikQuery -Path ./GetMSSQLDatabaseFromAvailabilityGroup.gql -QueryParams $QueryParms

if ($RSCMSSQLDatabase.logicalChildConnection.nodes.count -eq 0) {
    Write-Error "No database found with name: $DatabaseName, on AG: $AvailabilityGroupName"
}
$RSCMSSQLDatabase.logicalChildConnection.nodes


name               id
----               --
ProductionDatabase 02334333-d65b-584c-8a0d-cbe07a950f53
ProductionDatabase 9ce3902c-2318-58d0-aa8d-c366b395f63c
ProductionDatabase c3656bda-aeeb-5faa-a62a-71913fc3b713
ProductionDatabase c8a743d4-5683-514d-a593-2f30047768e3
ProductionDatabase f0cfde6e-a708-556d-ba14-ded265f94d2c



# Get the Latest Recovery Point

In [13]:
$QueryParms = @{
    "id" = "$($RSCMSSQLDatabase.logicalChildConnection.nodes[0].id)"
}
$RSCMSSQLRecoverableRange = Invoke-RubrikQuery -Path ./MssqlDatabaseDetailsRecoverableRangesQuery.gql -QueryParams $QueryParms | Sort-Object {$_.data.endTime}
$RSCMSSQLDatabaseLatestRecoveryPoint = ($RSCMSSQLRecoverableRange.data.endTime[-1]).ToUniversalTime().ToString('yyyy-MM-ddTHH:mm:ss.fffZ')
$RSCMSSQLDatabaseLatestRecoveryPoint

2023-05-26T15:05:53.000Z


# Get the Original DB File Locations

In [18]:
$QueryParms = @{
    "input" = @{
        "id" = "$($RSCMSSQLDatabase.logicalChildConnection.nodes[0].id)"
        "time" = $($RSCMSSQLDatabaseLatestRecoveryPoint)
    }
}
$RSCMSSQLFileLocations = Invoke-RubrikQuery -Path ./AllMssqlDatabaseRestoreFilesQuery.gql -QueryParams $QueryParms

$TargetFilePaths = @()

foreach ($DBfile in $RSCMSSQLFileLocations.items) {
    $originalName = [system.io.fileinfo]$DBFile.originalName
    $newFilename = $originalName.Basename + $FilenameAppend +  $originalName.Extension
    $targetFilePath = @{
        "newFilename" = $newFilename
        "exportPath" = $DBfile.originalPath
        "logicalName" = $DBFile.logicalName
    }
    $TargetFilePaths += $targetFilePath
}

$targetFilePaths


Name                           Value
----                           -----
exportPath                     c:\mnt\sqldata
logicalName                    AdventureWorks2017
newFilename                    ProductionDatabase_tongexport.mdf
exportPath                     c:\mnt\sqllogs
logicalName                    AdventureWorks2017_log
newFilename                    ProductionDatabase_Log_tongexport.ldf



# Export the Database to the Secondary Replicas

In [19]:
foreach ($SQLInstance in $SecondarySQLServerInstance){
    $QueryParms = @{
        "hostname" = "$($SQLInstance)"
        "instance" = "MSSQLSERVER"
    }
    $RSCMSQLInstance = Invoke-RubrikQuery -Path ./GetMSSQLInstance.gql -QueryParams $QueryParms
    $RSCMSQLInstance    

    $QueryParms = @{
        "input" = @{
            "id" = "$($RSCMSSQLDatabase.logicalChildConnection.nodes[0].id)"
            "config" = @{
                "recoveryPoint" = @{
                    "date" = "$($RSCMSSQLDatabaseLatestRecoveryPoint)"
                }
                "targetInstanceId" = "$($RSCMSQLInstance.physicalChildConnection.nodes.id)"
                "targetDatabaseName" = "$($TargetDatabaseName)"
                "targetDataFilePath" = ""
                "targetLogFilePath" = ""
                "targetFilePaths" = $TargetFilePaths
                "allowOverwrite" = $true
                "finishRecovery" = $false
            }
        }
    }
    Invoke-RubrikQuery -Path ./MssqlDatabaseExportMutation.gql -QueryParams $QueryParms
}
Start-Sleep -Seconds 45


id                                   name                           cbtStatus physicalChildConnecti
                                                                              on
--                                   ----                           --------- ---------------------
1176c065-3434-5c87-b69f-597560c8cb17 rp-sql19ags-1a.perf.rubrik.com Enabled   @{nodes=System.Objec…
6e02668f-2d63-5a2d-90bf-204df7676728 rp-sql19ags-1b.perf.rubrik.com Enabled   @{nodes=System.Objec…



# Export the Database to the Primary Replica

In [20]:
$QueryParms = @{
    "hostname" = "$($PrimarySQLServerInstance)"
    "instance" = "MSSQLSERVER"
}
$RSCMSQLInstance = Invoke-RubrikQuery -Path ./GetMSSQLInstance.gql -QueryParams $QueryParms
$RSCMSQLInstance    

$QueryParms = @{
    "input" = @{
        "id" = "$($RSCMSSQLDatabase.logicalChildConnection.nodes[0].id)"
        "config" = @{
            "recoveryPoint" = @{"date" = "$($RSCMSSQLDatabaseLatestRecoveryPoint)"}
            "targetInstanceId" = "$($RSCMSQLInstance.physicalChildConnection.nodes.id)"
            "targetDatabaseName" = "$($TargetDatabaseName)"
            "targetDataFilePath" = ""
            "targetLogFilePath" = ""
            "targetFilePaths" = $TargetFilePaths
            "allowOverwrite" = $true
            "finishRecovery" = $true
        }
    }
}
$req = Invoke-RubrikQuery -Path ./MssqlDatabaseExportMutation.gql -QueryParams $QueryParms
Start-Sleep -Seconds 45


id                                   name                           cbtStatus physicalChildConnecti
                                                                              on
--                                   ----                           --------- ---------------------
23fdb155-57bb-5cbb-9ec6-847bad6f09aa rp-sql19ags-1c.perf.rubrik.com Enabled   @{nodes=System.Objec…



# Add the Database into the AG on the Primary Replica

In [22]:
$Query = "ALTER AVAILABILITY GROUP [$($AvailabilityGroupName)] ADD DATABASE [$($TargetDatabaseName)];"
Invoke-DbaQuery -SqlInstance $PrimarySQLServerInstance -Query $Query

# Add the Database into the AG on Secondary Replicas

In [3]:
foreach ($SQLInstance in $SecondarySQLServerInstance){
    $Query = "ALTER DATABASE [$($TargetDatabaseName)] SET HADR AVAILABILITY GROUP = [$($AvailabilityGroupName)];"
    Invoke-DbaQuery -SqlInstance $SQLInstance -Query $Query
}